# RFM Analysis

### RFM factors illustrate these facts:

#### - the more recent the purchase, the more responsive the customer is to promotions
#### - the more frequently the customer buys, the more engaged and satisfied they are
#### - monetary value differentiates heavy spenders from low-value purchasers

In [ ]:
import findspark
findspark.init()

import pyspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
spark = SparkSession.builder.appName("RFM Analysis").master("local[*]").getOrCreate()

22/02/05 13:13:22 WARN Utils: Your hostname, EMPID21092 resolves to a loopback address: 127.0.1.1; using 192.168.1.6 instead (on interface wlp3s0)
22/02/05 13:13:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/05 13:13:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data = spark.read.format("csv")\
.options(header='true', inferschema='true', delimiter=',')\
.load("/home/rita/Documents/Spark/Assignments/data.csv")
data.show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 5 rows



In [4]:
data.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [5]:
data = data.withColumn("Quantity", data["Quantity"].cast(IntegerType()))
data = data.withColumn("UnitPrice", data["UnitPrice"].cast(DoubleType()))

In [6]:
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY") #adding this resolved SparkUpgradeException
data = data.withColumn("Date", to_date(unix_timestamp("InvoiceDate", "MM/dd/yyyy").cast("timestamp")))
data.show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|      Date|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|2010-12-01|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-12-01|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|2010-12-01|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-12-01|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-12-01|
+---------+---------+--------------------+--------+--------------+---------+----------+-

In [7]:
data = data.withColumn("Total", round(data["UnitPrice"] * data["Quantity"], 2))

In [8]:
data.show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+-----+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|      Date|Total|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+-----+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|2010-12-01| 15.3|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-12-01|20.34|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|2010-12-01| 22.0|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-12-01|20.34|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-12-01|20.34|
+---------+---------+-------------------

In [40]:
#spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY") #adding this resolved SparkUpgradeException
data = data.withColumn("RecencyDays", expr("datediff('2011-12-31', Date)"))

In [41]:
data.show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+-----+-----------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|      Date|Total|RecencyDays|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+-----+-----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|2010-12-01| 15.3|        395|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-12-01|20.34|        395|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|2010-12-01| 22.0|        395|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-12-01|20.34|        395|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|

In [43]:
rfm_table = data.groupBy("CustomerId")\
                        .agg(min("RecencyDays").alias("Recency"), \
                             count("InvoiceNo").alias("Frequency"), \
                             round(sum("Total"),2).alias("Monetary"))
rfm_table.show(5)

+----------+-------+---------+--------+
|CustomerId|Recency|Frequency|Monetary|
+----------+-------+---------+--------+
|     17420|     72|       30|  598.83|
|     16861|     81|        8|  151.65|
|     16503|    128|       86| 1421.43|
|     15727|     38|      302| 5178.96|
|     17389|     22|      224|31300.08|
+----------+-------+---------+--------+
only showing top 5 rows



In [68]:
r_quartile = rfm_table.approxQuantile("Recency", [0.2, 0.5, 0.75, 0.9], 0)
f_quartile = rfm_table.approxQuantile("Frequency", [0.2, 0.5, 0.75, 0.9], 0)
m_quartile = rfm_table.approxQuantile("Monetary", [0.2, 0.5, 0.75, 0.9], 0)
print(r_quartile,f_quartile,m_quartile)

[33.0, 72.0, 165.0, 285.0] [14.0, 42.0, 102.0, 212.0] [234.34, 648.41, 1612.13, 3515.68]


In [69]:
rfm_table = rfm_table.withColumn("R_Quartile", \
                                 when(col("Recency") >= r_quartile[3] , 1).\
                                 when(col("Recency") >= r_quartile[2] , 2).\
                                 when(col("Recency") >= r_quartile[1] , 3).\
                                 when(col("Recency") >= r_quartile[0] , 4).\
                                 otherwise(5))
rfm_table.show(5)

+----------+-------+---------+--------+----------+----------+----------+---------+
|CustomerId|Recency|Frequency|Monetary|R_Quartile|F_Quartile|M_Quartile|RFM_Score|
+----------+-------+---------+--------+----------+----------+----------+---------+
|     17420|     72|       30|  598.83|         3|         2|         2|      222|
|     16861|     81|        8|  151.65|         3|         1|         1|      211|
|     16503|    128|       86| 1421.43|         3|         3|         3|      233|
|     15727|     38|      302| 5178.96|         4|         4|         4|      344|
|     17389|     22|      224|31300.08|         5|         4|         4|      444|
+----------+-------+---------+--------+----------+----------+----------+---------+
only showing top 5 rows



In [73]:
rfm_table = rfm_table.withColumn("F_Quartile", \
                                 when(col("Frequency") >= f_quartile[3] , 5).\
                                 when(col("Frequency") >= f_quartile[2] , 4).\
                                 when(col("Frequency") >= f_quartile[1] , 3).\
                                 when(col("Frequency") >= f_quartile[0] , 2).\
                                 otherwise(1))
rfm_table.show(5)

+----------+-------+---------+--------+----------+----------+----------+---------+
|CustomerId|Recency|Frequency|Monetary|R_Quartile|F_Quartile|M_Quartile|RFM_Score|
+----------+-------+---------+--------+----------+----------+----------+---------+
|     17420|     72|       30|  598.83|         3|         2|         2|      222|
|     16861|     81|        8|  151.65|         3|         1|         1|      211|
|     16503|    128|       86| 1421.43|         3|         3|         3|      233|
|     15727|     38|      302| 5178.96|         4|         5|         4|      344|
|     17389|     22|      224|31300.08|         5|         5|         4|      444|
+----------+-------+---------+--------+----------+----------+----------+---------+
only showing top 5 rows



In [74]:
rfm_table = rfm_table.withColumn("M_Quartile", \
                                 when(col("Monetary") >= m_quartile[3] , 5).\
                                 when(col("Monetary") >= m_quartile[2] , 4).\
                                 when(col("Monetary") >= m_quartile[1] , 3).\
                                 when(col("Monetary") >= m_quartile[0] , 2).\
                                 otherwise(1))
rfm_table.show(5)

+----------+-------+---------+--------+----------+----------+----------+---------+
|CustomerId|Recency|Frequency|Monetary|R_Quartile|F_Quartile|M_Quartile|RFM_Score|
+----------+-------+---------+--------+----------+----------+----------+---------+
|     17420|     72|       30|  598.83|         3|         2|         2|      222|
|     16861|     81|        8|  151.65|         3|         1|         1|      211|
|     16503|    128|       86| 1421.43|         3|         3|         3|      233|
|     15727|     38|      302| 5178.96|         4|         5|         5|      344|
|     17389|     22|      224|31300.08|         5|         5|         5|      444|
+----------+-------+---------+--------+----------+----------+----------+---------+
only showing top 5 rows



In [103]:
rfm_table = rfm_table.withColumn("RFM_Score", round((col("R_Quartile")+col("F_Quartile")+col("M_Quartile"))/3,2))
rfm_table.show(5)

+----------+-------+---------+--------+----------+----------+----------+---------+
|CustomerId|Recency|Frequency|Monetary|R_Quartile|F_Quartile|M_Quartile|RFM_Score|
+----------+-------+---------+--------+----------+----------+----------+---------+
|     17420|     72|       30|  598.83|         3|         2|         2|     2.33|
|     16861|     81|        8|  151.65|         3|         1|         1|     1.67|
|     16503|    128|       86| 1421.43|         3|         3|         3|      3.0|
|     15727|     38|      302| 5178.96|         4|         5|         5|     4.67|
|     17389|     22|      224|31300.08|         5|         5|         5|      5.0|
+----------+-------+---------+--------+----------+----------+----------+---------+
only showing top 5 rows



In [104]:
rfm_table.select("CustomerID").where("RFM_Score >= 5.0").show(5)

+----------+
|CustomerID|
+----------+
|     17389|
|     12471|
|     13468|
|     16549|
|     17677|
+----------+
only showing top 5 rows



In [105]:
grouped_by_rfmscore = rfm_table.groupBy("RFM_Score").count().orderBy("count", ascending=False)
grouped_by_rfmscore_pandas = grouped_by_rfmscore.toPandas()
grouped_by_rfmscore_pandas.head(5)

,RFM_Score,count
0,2.33,497
1,3.00,487
2,2.67,474
3,2.00,469
4,3.33,421


In [106]:
#!pip install plotly

In [107]:
import plotly.graph_objects as go

data = [go.Bar(x=grouped_by_rfmscore_pandas['RFM_Score'], y=grouped_by_rfmscore_pandas['count'])]

layout = go.Layout(
    title=go.layout.Title(
        text='Customer RFM Segments'
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text='RFM Segment'
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='Number of Customers'
        )
    )
)

fig = go.Figure(data=data, layout=layout)
fig.show()

22/02/05 21:28:38 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1347867 ms exceeds timeout 120000 ms
22/02/05 21:28:38 WARN SparkContext: Killing executors is not supported by current scheduler.
